<a href="https://colab.research.google.com/github/mehdimsv/Deep-Reinforcement-Learning-Hands-On-Second-Edition/blob/master/gold_price_prediction_DeepNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Sep 29 13:46:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
try:
  from tqdm import tqdm
except :
  !pip install --quiet tqdm==4.59.0
  from tqdm import tqdm

In [ ]:
dir_path="gdrive/My Drive/ML4T/gold_prediction"
file_name="XAUUSD15_final.csv"

In [ ]:
import os
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
print(os.getcwd())
!ls

/content
gdrive	sample_data


In [ ]:
!cd gdrive
!ls


gdrive	sample_data


In [ ]:
if not os.getcwd().endswith("gold_prediction"):
  os.chdir(dir_path)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv(file_name)
#df.set_index("date",inplace=True)

In [ ]:
df.head()

,date,day_of_month_1,day_of_month_2,day_of_month_3,day_of_month_4,day_of_month_5,day_of_month_6,day_of_month_7,day_of_month_8,day_of_month_9,day_of_month_10,day_of_month_11,day_of_month_12,day_of_month_13,day_of_month_14,day_of_month_15,day_of_month_16,day_of_month_17,day_of_month_18,day_of_month_19,day_of_month_20,day_of_month_21,day_of_month_22,day_of_month_23,day_of_month_24,day_of_month_25,day_of_month_26,day_of_month_27,day_of_month_28,day_of_month_29,day_of_month_30,day_of_month_31,day_0,day_1,day_2,day_3,day_4,day_6,hour_0,hour_1,...,percent_avg_candle_size_9,percent_avg_candle_size_20,candle_schema_0,candle_schema_1,candle_schema_2,candle_schema_3,candle_schema_4,candle_schema_5,candle_schema_6,candle_schema_7,candle_schema_8,candle_schema_9,sma5_-1,sma5_1,sma14_-1,sma14_1,sma21_-1,sma21_1,ema5_-1,ema5_1,ema14_-1,ema14_1,ema200_-1,ema200_1,mid_-1,mid_1,upper_band_-1,upper_band_1,lower_band_-1,lower_band_1,candle_type_-1,candle_type_0,candle_type_1,intersect_with_previous_candle,current_label_-1,current_label_0,current_label_1,label_-1,label_0,label_1
0,2012-10-25 00:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,1,1,0,0,0.66,0,1,0,0,1,0
1,2012-10-25 00:45:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,1,1,0,0,0.80,1,0,0,0,1,0
2,2012-10-25 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,1,1,0,0,1,1,1,1,1,1,0,0,1,0,0,1,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0.43,1,0,0,0,1,0
3,2012-10-25 01:15:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,1,0.88,0,0,1,0,1,0
4,2012-10-25 01:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,1,1,0,0,0.93,0,1,0,0,1,0


##Convert label categories to one column

In [ ]:
cols=list(df.columns.values[1:])
data_df=df.loc[:,cols]

In [ ]:
print(data_df.columns.values)

['day_of_month_1' 'day_of_month_2' 'day_of_month_3' 'day_of_month_4'
 'day_of_month_5' 'day_of_month_6' 'day_of_month_7' 'day_of_month_8'
 'day_of_month_9' 'day_of_month_10' 'day_of_month_11' 'day_of_month_12'
 'day_of_month_13' 'day_of_month_14' 'day_of_month_15' 'day_of_month_16'
 'day_of_month_17' 'day_of_month_18' 'day_of_month_19' 'day_of_month_20'
 'day_of_month_21' 'day_of_month_22' 'day_of_month_23' 'day_of_month_24'
 'day_of_month_25' 'day_of_month_26' 'day_of_month_27' 'day_of_month_28'
 'day_of_month_29' 'day_of_month_30' 'day_of_month_31' 'day_0' 'day_1'
 'day_2' 'day_3' 'day_4' 'day_6' 'hour_0' 'hour_1' 'hour_2' 'hour_3'
 'hour_4' 'hour_5' 'hour_6' 'hour_7' 'hour_8' 'hour_9' 'hour_10' 'hour_11'
 'hour_12' 'hour_13' 'hour_14' 'hour_15' 'hour_16' 'hour_17' 'hour_18'
 'hour_19' 'hour_20' 'hour_21' 'hour_22' 'hour_23' 'min30' 'momentum5'
 'momentum14' 'macd' 'macdsignal' 'macdhist' 'bbp' 'atr5' 'atr14' 'adx5'
 'adx14' 'percent_of_body' 'percent_avg_candle_size_9'
 'percent_avg

## 

In [ ]:
NUM_CLASSES=3
percent=0.85
train_size=int(data_df.shape[0]*percent)
x_train=data_df[:train_size]
x_test=data_df[train_size+1:]
x_train=x_train.dropna()
#x_train=normalize_data(x_train)
x_test=x_test.dropna()
#x_test=normalize_data(x_test)
x_test=x_test.reset_index()

x_train=x_train.values
x_test=x_test.values

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter


In [ ]:
class FeatureDataset(Dataset):
  def __init__(self,data):
    self.data=data

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    features=self.data[idx,:-3]
    labels=self.data[idx,-3:]
    return torch.Tensor(features),torch.from_numpy(labels).long()

In [ ]:
N_EPOCHS=250
BATCH_SIZE=64
train_dataloader=DataLoader(
    FeatureDataset(x_train),
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=True

)
test_dataloader=DataLoader(
    FeatureDataset(x_test),
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=True
)

## Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
class DeepModel(nn.Module):
  def __init__(self,batch_size,n_features,n_classes=3,dropout=0.7):
    super(DeepModel,self).__init__()
    self.n_classes=n_classes
    self.fc1=nn.Linear(n_features,1024)
    self.dropout=nn.Dropout(dropout)
    self.fc2=nn.Linear(n_features,512)
    self.dropout2=nn.Dropout(dropout)
    self.classifier=nn.Linear(512,n_classes)

  def forward(self,x):
    print("x shape:",x.shape)
    x=F.relu(self.fc1(x))
    #x=self.dropout(x)
    x=F.relu(self.fc2(x))
    #x=self.dropout(x)
    print("x shape:",x.shape)
    x= self.classifier(x)
    print("result :",x.shape)
    return F.log_softmax(x,dim=1)

## Model Initialization

In [ ]:
#model=PricePredictionModel(BATCH_SIZE,data_df.shape[1])
model=DeepModel(BATCH_SIZE,data_df.shape[1]-3)
model=model.to(device)
#odel.hidden_state=model.init_hidden(BATCH_SIZE)
weights = torch.tensor([1.5,0.1,1.5]).to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.005)

In [ ]:
print(model)

DeepModel(
  (fc1): Linear(in_features=110, out_features=1024, bias=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc2): Linear(in_features=110, out_features=512, bias=True)
  (dropout2): Dropout(p=0.7, inplace=False)
  (classifier): Linear(in_features=512, out_features=3, bias=True)
)


## Train & Test

In [ ]:
def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

In [ ]:
def train(dataloader,model,loss_fn,optimizer,epoch):
  #size=len(dataloader.dataset)
  #torch.autograd.set_detect_anomaly(True)
  #hidden = model.init_hidden(BATCH_SIZE)
  model.train()
  running_loss=0.0
  running_correct=0.0
  for batch,(X,y) in enumerate(dataloader):
    if batch==2:
      break
    X=X.to(device)
    y=y.to(device)
    optimizer.zero_grad()
    #compute prediction Error
    #hidden=(hidden[0].detach(),hidden[1].detach())
    pred=model(X)
    print("pred shape :",pred.shape)
    print("--------")
    print("y shape:",y.shape)
    pred = torch.squeeze(pred)
    print("pred shape:",pred.shape)
    loss=loss_fn(pred,y)
    step_accuracy=accuracy(torch.argmax(y.detach().to("cpu"),dim=1),torch.argmax(pred.detach().to("cpu"),dim=1))

    #Backpropagation
    
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
    running_correct+=(torch.argmax(pred.detach(),dim=1)==torch.argmax(y,dim=1)).type(torch.float).sum().item()
    if batch % 100==0:
      loss,current=loss.item(),batch*len(X)
      print(f"loss: {loss:>7f}   [{current:>5d}/{size:>5d}], accuracy:{step_accuracy*100}%")
  running_loss/=size
  running_correct/=size
  writer.add_scalar("Loss/train",running_loss , epoch)
  writer.add_scalar("Accuracy/train",running_correct, epoch)
  print(f"Average Loss: {running_loss:>7f}   ,Avg accuracy:{running_correct*100}%")
  writer.flush()

In [ ]:
def test(dataloader,model,epoch):
  size=len(dataloader.dataset)
  model.eval()
  test_loss,correct=0,0
  with torch.no_grad():
    for X,y in dataloader:
      X=X.to(device)
      y=y.to(device)
      pred=model(X)
     
      test_loss+=loss_fn(pred,y).item()
      #step_accuracy=accuracy_score(y.detach().to("cpu"),torch.argmax(pred.detach().to("cpu"),dim=1))
      correct+=(torch.argmax(pred.detach(),dim=1)==y).type(torch.float).sum().item()

  
  test_loss/=size
  correct/=size
  writer.add_scalar("Loss/test", test_loss,epoch)
  writer.add_scalar("Accuracy/test", correct,epoch)
  print(f"Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss:{test_loss:>8f}\n")
  writer.flush()

## Loading TensorBoard

## Training Loop

In [ ]:
epochs=250
for epoch in range(epochs):
  print(f"Epoch {epoch+1} \n---------------------------------------")
  train(train_dataloader,model,loss_fn,optimizer,epoch)
  test(test_dataloader,model,epoch)
  torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_fn
            }, "gold-price-prediction-model-resume.pth")
print("Done!")

Epoch 1 
---------------------------------------
x shape: torch.Size([64, 110])
x shape: torch.Size([64, 512])
result : torch.Size([64, 3])
pred shape : torch.Size([64, 3])
--------
y shape: torch.Size([64, 3])
pred shape: torch.Size([64, 3])


RuntimeError: ignored